Learn how to use Imagefolder
<br>Scheduler -> to change the learning rate
<br>Transfer Learning

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import os
import copy

Device Configuration

In [9]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
mean=np.array([0.0485,0.456,0.406])
std=np.array([0.229,0.224,0.225])

data_transforms={
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
}

In [11]:
data_dir='hymenoptera_data'
sets=['train','val']
image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),
                                      data_transforms[x]) for x in ['train','val']}
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: hymenoptera_data\train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.0485 0.456  0.406 ], std=[0.229 0.224 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: hymenoptera_data\val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.0485 0.456  0.406 ], std=[0.229 0.224 0.225])
            )}

In [12]:
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=0) for x in ['train','val']
}
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x257ab6be490>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x257af8d9890>}

In [13]:
class_names=image_datasets['train'].classes
class_names

['ants', 'bees']

In [14]:
data_sizes={x:len(image_datasets[x]) for x in ['train','val']}

In [19]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=25):
    since=time.time()
    best_model_wts=copy.deepcopy(model.state_dict())
    best_acc=0.0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs}')
        print('-'*10)
        for phase in ['train','val']:
            if phase=='train':
                model.train()
            else:
                model.eval()
            running_loss=0.0
            running_corrects=0
            
            for inputs,labels in dataloaders[phase]:
                inputs=inputs.to(device)
                labels=labels.to(device)
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs=model(inputs)
                    _,preds=torch.max(outputs,1)
                    loss=criterion(outputs,labels)
                    if phase=='train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                running_loss+=loss.item()*inputs.size(0)
                running_corrects+=torch.sum(preds==labels.data)
                if phase=='train':
                    scheduler.step()
            epoch_loss=running_loss/data_sizes[phase]
            epoch_acc=running_corrects.double()/data_sizes[phase]

            print('{} Loss: {:.4f} Acc : {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if phase=='val' and epoch_acc>best_acc:
                best_acc=epoch_acc
                best_model_wts=copy.deepcopy(model.state_dict())
            
            print()
        
        time_elapsed=time.time()-since
        print('Training comlete in {:.0f}m {:.0f}s'.format(time_elapsed//60,time_elapsed%60))
        print('Best val Accuracy:{:.4f}'.format(best_acc))
        model.load_state_dict(best_model_wts)
    return model

In [20]:
model=models.resnet18(pretrained=True)
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,2)
model=model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001)

In [21]:
step_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

In [22]:
model=train_model(model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

Epoch 0/25
----------
train Loss: 0.6991 Acc : 0.5697

val Loss: 0.6530 Acc : 0.5882

Training comlete in 0m 46s
Best val Accuracy:0.5882
Epoch 1/25
----------
train Loss: 0.7169 Acc : 0.5246

val Loss: 0.6761 Acc : 0.5752

Training comlete in 1m 41s
Best val Accuracy:0.5882
Epoch 2/25
----------
train Loss: 0.6607 Acc : 0.5943

val Loss: 0.6478 Acc : 0.6078

Training comlete in 2m 27s
Best val Accuracy:0.6078
Epoch 3/25
----------
train Loss: 0.6690 Acc : 0.5902

val Loss: 0.6624 Acc : 0.6013

Training comlete in 3m 19s
Best val Accuracy:0.6078
Epoch 4/25
----------
train Loss: 0.6846 Acc : 0.5574

val Loss: 0.6530 Acc : 0.6209

Training comlete in 4m 37s
Best val Accuracy:0.6209
Epoch 5/25
----------
train Loss: 0.6656 Acc : 0.5779

val Loss: 0.6594 Acc : 0.5752

Training comlete in 5m 36s
Best val Accuracy:0.6209
Epoch 6/25
----------
train Loss: 0.6835 Acc : 0.5697

val Loss: 0.6424 Acc : 0.6340

Training comlete in 6m 25s
Best val Accuracy:0.6340
Epoch 7/25
----------
train Loss: 

In [23]:
model=models.resnet18(pretrained=True)
for params in model.parameters():
    params.requires_grad=False
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,2)
model=model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=0.001)
step_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)
model=train_model(model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

Epoch 0/25
----------
train Loss: 0.6684 Acc : 0.5615

val Loss: 0.6433 Acc : 0.6078

Training comlete in 0m 26s
Best val Accuracy:0.6078
Epoch 1/25
----------
train Loss: 0.6580 Acc : 0.6311

val Loss: 0.6407 Acc : 0.6013

Training comlete in 0m 58s
Best val Accuracy:0.6078
Epoch 2/25
----------
train Loss: 0.6684 Acc : 0.6148

val Loss: 0.6410 Acc : 0.6078

Training comlete in 1m 33s
Best val Accuracy:0.6078
Epoch 3/25
----------
train Loss: 0.6607 Acc : 0.6148

val Loss: 0.6557 Acc : 0.6144

Training comlete in 2m 0s
Best val Accuracy:0.6144
Epoch 4/25
----------
train Loss: 0.6543 Acc : 0.6311

val Loss: 0.6457 Acc : 0.6144

Training comlete in 2m 31s
Best val Accuracy:0.6144
Epoch 5/25
----------
train Loss: 0.6557 Acc : 0.6516

val Loss: 0.6572 Acc : 0.5621

Training comlete in 3m 0s
Best val Accuracy:0.6144
Epoch 6/25
----------
train Loss: 0.6507 Acc : 0.6230

val Loss: 0.6435 Acc : 0.6013

Training comlete in 3m 26s
Best val Accuracy:0.6144
Epoch 7/25
----------
train Loss: 0.